In [7]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel('backend/classification/TrainingData_de.xlsx')

print("Column headings:")
print(df.columns)

Column headings:
Index(['Unnamed: 0', 'platform', 'User', 'topic', 'phase/arg_situation',
       'method', 'adressee, reference', 'function', 'intent', 'Effekt',
       'impl_expl', 'pos_neg', 'utterance', 'easy translation', 'Unnamed: 14',
       'Unnamed: 15'],
      dtype='object')


In [9]:
from collections import Counter, OrderedDict

category_names = Counter()
for i in df.index:
    category_names[df['Effekt'][i]] +=1

for entry in category_names.most_common():
    print(entry)


('discreditation', 146)
('lecture', 126)
('revelation', 72)
('support', 64)
('insinuation', 64)
('indication', 56)
('claim', 54)
('attack', 53)
('dissence', 33)
('praise', 32)
('accusation', 29)
('colonisation', 26)
('fault', 26)
('speculation', 26)
('judgement', 22)
('challenge', 17)
('warning', 15)
('statement', 15)
('gratitude', 14)
('sarcasm', 13)
('trolling', 13)
('demand', 12)
('ambivalence', 11)
('advice', 10)
('question', 9)
('direction', 9)
('solution', 9)
('concession', 8)
('relativation', 7)
('reflection', 7)
('irony', 5)
('bragging', 4)
('conspiracy', 4)
('provocation', 3)
('confrontation', 3)
('whataboutism', 2)
('consent', 2)
('link', 1)
('humour', 1)
